# Overview

In this notebook, we will use our own dataset as the knwoloedge of the RAG. We will load our data from CSV file. Next, loading `RagRetriever` witht the customise data. Finally, we will do some inference under the context of the customize data.


# Installing faiss(GPU+CPU)

Here we install Faceboook Faiss which is a library for efficient **similarity search** and **cluster of dense vectors**. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. Some of the most useful algorithms are implemented on the GPU. Further more, the GPU implementation can accept input from either CPU or GPU memeory. 

In [1]:
%%capture
!conda install -c pytorch -c nvidia faiss-gpu=1.8.0 -y

In [2]:
# for checking faiss-gpu
import faiss

# Loading Dataset

The dataset needed for RAG must have two columns:
- title(string): title of the document
- text(string): text of a passage of the document

In [3]:
import pandas as pd

csv_path='/kaggle/input/knowledge-dataset/own_knwoledge_dataset.csv'

df=pd.read_csv(csv_path)
print(df.shape)

(3, 2)
